In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

# -- 1. ЗАГРУЗКА ДАННЫХ --
# Загружаем "чистые" данные
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# Сохраняем PassengerId для файла отправки
test_passenger_ids = test_df['PassengerId']

# ОБЪЕДИНЯЕМ train и test в один датафрейм (full_df)
# Это "золотой стандарт" для feature engineering.
# Он гарантирует, что мы обработаем оба набора данных АБСОЛЮТНО одинаково.
full_df = pd.concat([train_df.drop('Survived', axis=1), test_df], axis=0)

print(f"Размер Train: {train_df.shape}")
print(f"Размер Test: {test_df.shape}")
print(f"Размер Full: {full_df.shape}")

Размер Train: (891, 12)
Размер Test: (418, 11)
Размер Full: (1309, 11)


In [2]:
# --- 2. FEATURE ENGINEERING ---
# Здесь мы создаем ВСЕ фичи, которые нашли (наши + из референса)

# -- ЗАПОЛНЕНИЕ ПРОПУСКОВ (Базовое) --
# Заполняем Age и Fare медианой, Embarked - модой.
full_df['Age'] = full_df['Age'].fillna(full_df['Age'].median())
full_df['Fare'] = full_df['Fare'].fillna(full_df['Fare'].median())
full_df['Embarked'] = full_df['Embarked'].fillna(full_df['Embarked'].mode()[0])

# -- ФИЧА 1: Title (Титул) --
full_df['Title'] = full_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# Группируем редкие титулы
common_titles = ['Mr', 'Miss', 'Mrs', 'Master']
full_df['Title'] = full_df['Title'].apply(lambda x: x if x in common_titles else 'Rare')
print("Фича 'Title' создана.")

# -- ФИЧА 2: FamilySize (Размер семьи) и IsAlone (Одиночка) --
full_df['FamilySize'] = full_df['SibSp'] + full_df['Parch'] + 1
full_df['IsAlone'] = (full_df['FamilySize'] == 1).astype(int)
print("Фичи 'FamilySize' и 'IsAlone' созданы.")

# -- ФИЧА 3: Deck (Палуба) --
# Извлекаем букву палубы из 'Cabin'. Если NaN, ставим 'U' (Unknown)
full_df['Deck'] = full_df['Cabin'].apply(lambda x: str(x)[0] if pd.notna(x) else 'U')
print("Фича 'Deck' создана.")

# -- ФИЧА 4: FarePerPerson (Цена на человека) --
full_df['FarePerPerson'] = full_df['Fare'] / full_df['FamilySize']
print("Фича 'FarePerPerson' создана.")

# -- ФИЧА 5: Age*Class (Возраст * Класс) --
# Взаимодействие фичей: ребенок в 1-м классе - не то же самое, что в 3-м
full_df['Age*Class'] = full_df['Age'] * full_df['Pclass']
print("Фича 'Age*Class' создана.")

# -- ПОДГОТОВКА К МОДЕЛИ --

# -- УДАЛЕНИЕ "МУСОРНЫХ" КОЛОНОК --
# Они нам больше не нужны, т.к. мы извлекли из них всю пользу
drop_cols = ['Name', 'Ticket', 'Cabin', 'PassengerId']
full_df = full_df.drop(columns=drop_cols)

# -- ПРЕВРАЩЕНИЕ ТЕКСТА В ЦИФРЫ (ПРАВИЛЬНЫЙ СПОСОБ: One-Hot Encoding) --
# Мы НЕ используем map(). Мы используем pd.get_dummies()
# Это превратит 'Title_Mr', 'Title_Miss' и т.д. в колонки 0/1
# Это также превратит 'Sex', 'Embarked', 'Deck' и 'Title'
categorical_features = ['Sex', 'Embarked', 'Title', 'Deck']
full_df = pd.get_dummies(full_df, columns=categorical_features, drop_first=True)

print("\n--- Финальный датафрейм готов к масштабированию ---")
full_df.head()

Фича 'Title' создана.
Фичи 'FamilySize' и 'IsAlone' созданы.
Фича 'Deck' создана.
Фича 'FarePerPerson' создана.
Фича 'Age*Class' создана.

--- Финальный датафрейм готов к масштабированию ---


,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,FarePerPerson,Age*Class,Sex_male,...,Title_Mrs,Title_Rare,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_U
0,3,22.0,1,0,7.2500,2,0,3.62500,66.0,True,...,False,False,False,False,False,False,False,False,False,True
1,1,38.0,1,0,71.2833,2,0,35.64165,38.0,False,...,True,False,False,True,False,False,False,False,False,False
2,3,26.0,0,0,7.9250,1,1,7.92500,78.0,False,...,False,False,False,False,False,False,False,False,False,True
3,1,35.0,1,0,53.1000,2,0,26.55000,35.0,False,...,True,False,False,True,False,False,False,False,False,False
4,3,35.0,0,0,8.0500,1,1,8.05000,105.0,True,...,False,False,False,False,False,False,False,False,False,True


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# --- 3. МАСШТАБИРОВАНИЕ И МОДЕЛИРОВАНИЕ ---

# -- МАСШТАБИРОВАНИЕ (Как в референсе) --
# Мы "приводим к одному масштабу" ВСЕ наши числовые колонки
scaler = StandardScaler()
# Обучаем scaler (он "запоминает" среднее и std)
scaler.fit(full_df) 
# Трансформируем (применяем)
full_df_scaled = scaler.transform(full_df)

# Восстанавливаем DataFrame с правильными именами колонок
full_df = pd.DataFrame(full_df_scaled, columns=full_df.columns)
print("Данные отмасштабированы (StandardScaler).")

# -- РАЗДЕЛЕНИЕ ДАННЫХ ОБРАТНО НА TRAIN И TEST --
# Мы знаем, что в train_df было 891 строк
X_train = full_df.iloc[:len(train_df)]
X_test = full_df.iloc[len(train_df):]
y_train = train_df['Survived'] # y_train - это ОРИГИНАЛЬНАЯ колонка Survived

print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")
print(f"Размер y_train: {y_train.shape}")

# -- ОБУЧЕНИЕ ЛУЧШЕЙ МОДЕЛИ --
# Мы берем RandomForest (наш чемпион 78.5%) и "кормим" его НОВЫМИ,
# ИДЕАЛЬНО подготовленными данными.
# Мы используем ЛУЧШИЕ настройки (гиперпараметры) из референс-проектов,
# которые часто дают >80%
model = RandomForestClassifier(
    n_estimators=1000,      # Больше деревьев
    max_depth=7,            # Чуть глубже, чем 5
    min_samples_leaf=3,     # Тот же хороший параметр
    random_state=42         # 42 - стандартное "счастливое число" в ML
)

model.fit(X_train, y_train)
print("Финальная модель RandomForest ОБУЧЕНА!")



Данные отмасштабированы (StandardScaler).
Размер X_train: (891, 24)
Размер X_test: (418, 24)
Размер y_train: (891,)
Финальная модель RandomForest ОБУЧЕНА!


In [4]:
# -- СОЗДАНИЕ ФАЙЛА ДЛЯ ОТПРАВКИ --
predictions = model.predict(X_test).astype(int)

submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': predictions
})

# Сохраняем финальный файл
submission.to_csv('submission_CHAMPION_V2.csv', index=False)
print("\n--- ФИНАЛЬНЫЙ ФАЙЛ 'submission_CHAMPION_V2.csv' СОЗДАН! ---")
submission.head()


--- ФИНАЛЬНЫЙ ФАЙЛ 'submission_CHAMPION_V2.csv' СОЗДАН! ---


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
